# Partial Order Dynamic Time Warping

In [ ]:
#| default_exp podtw

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import os
import json
from frozendict import frozendict
from collections import defaultdict

from pydantic import BaseModel, ConfigDict
from typing import List, Any, Dict, Callable,Set, Optional

import numpy as np
import itertools as it
import re
import asyncio

from constraint import Problem,FunctionConstraint
from bidict import bidict

import logging 
from stringdale.core import checkLogs
from stringdale.mappings import access_object, parse_edge_descriptor


In [ ]:
#| export
logger = logging.getLogger(__name__)

## Mapping labels to fresh variables for CSP

In [ ]:
#| export
def int_to_excel_col(n):
    if n < 0:
        raise ValueError("Number must be non-negative")
    
    result = ""
    n += 1  # Adjust because Excel columns start at 1, not 0
    
    while n > 0:
        n -= 1  # Adjust for 0-based indexing
        result = chr(n % 26 + ord('A')).lower() + result
        n //= 26
        
    return result

In [ ]:

assert int_to_excel_col(0) == "a"
assert int_to_excel_col(25) == "z"
assert int_to_excel_col(26) == "aa"
assert int_to_excel_col(27) == "ab"
assert int_to_excel_col(51) == "az"
assert int_to_excel_col(52) == "ba"
assert int_to_excel_col(701) == "zz"
assert int_to_excel_col(702) == "aaa"


In [ ]:
#| export
class LabelToVar():
    def __init__(self):
        self.label_to_var = bidict()
        self.label_to_index = bidict()

    def add_label(self,label:str,idx:int):
        self.label_to_var[label] = int_to_excel_col(idx)
        self.label_to_index[label] = idx

    def get_label(self,col:str) -> str:
        return self.label_to_var.inverse[col]

    def get_index(self,label:str) -> int:
        return self.label_to_index[label]

    def get_col(self,label:str) -> int:
        return self.label_to_var[label]


In [ ]:
label_to_var = LabelToVar()
label_to_var.add_label("x",0)
label_to_var.add_label("y",1)
label_to_var.add_label("z",2)

assert label_to_var.get_col("x") == "a"
assert label_to_var.get_col("y") == "b"
assert label_to_var.get_col("z") == "c"

assert label_to_var.get_label("a") == "x"
assert label_to_var.get_label("b") == "y"
assert label_to_var.get_label("c") == "z"

assert label_to_var.get_index("x") == 0
assert label_to_var.get_index("y") == 1
assert label_to_var.get_index("z") == 2

## Mock comparison functions

In [ ]:
#| export

async def word_overlap(result: str, expected: str,**kwargs) -> float:
    """
    Calculate the distance between result and expected strings based on word overlap.
    Returns a value between 0 and 1, where:
    - 0 means perfect match (all words from result are in expected)
    - 1 means no overlap (no words from result are in expected)
    
    Args:
        result (str): The string to check words from
        expected (str): The string to check words against
        
    Returns:
        float: Distance metric between 0 and 1
    """
    if not isinstance(result,str) or not isinstance(expected,str):
        return np.inf
    # Convert both strings to lowercase and split into words
    result_words = set(result.lower().split())
    expected_words = set(expected.lower().split())
    
    # If result is empty, return 1.0 (maximum distance)
    if not result_words:
        return 1.0
    
    # Calculate overlap
    overlap = len(result_words.intersection(expected_words))
    total = len(result_words)
    
    # Calculate distance (1 - percentage)
    distance = 1.0 - (overlap / total)
    
    return distance

In [ ]:
# Example 1
result = "The quick brown fox"
expected = "The lazy brown dog"
assert await word_overlap(result, expected) == 0.5  # Output: 0.5 (2 out of 4 words match)

# Example 2
result = "Hello world"
expected = "Hello there world"
assert await word_overlap(result, expected) == 0.0  # Output: 0.0 (all words match)

# Example 3
result = "Python programming"
expected = "Java development"
assert await word_overlap(result, expected) == 1.0  # Output: 1.0 (no words match)

In [ ]:
#| export
def regex(out: str, expected: str,**kwargs) -> float:
    """
    Compare a string against a regex pattern.
    Returns 0 if the regex matches, 1 if it doesn't.
    
    Args:
        out (str): The string to check
        expected (str): The regex pattern to match against
        
    Returns:
        float: 0 if match, 1 if no match
    """
    if not isinstance(out,str) or not isinstance(expected,str):
        return np.inf
    try:
        if re.search(expected, out,flags=re.IGNORECASE) is not None:
            return 0.0
        return 1.0
    except Exception:
        return 1.0


In [ ]:
# Basic matching
assert regex("hello world", "hello") == 0.0  # Simple substring match
assert regex("hello world", "^hello") == 0.0  # Start anchor
assert regex("hello world", "world$") == 0.0  # End anchor
assert regex("hello world", "hello.*world") == 0.0  # Pattern with wildcard

# Non-matching
assert regex("hello world", "goodbye") == 1.0  # No match
assert regex("hello world", "^world") == 1.0  # Wrong position
assert regex("hello world", "hello$") == 1.0  # Wrong position with anchor

# Pattern errors and edge cases
assert regex("hello world", "(unclosed") == 1.0  # Invalid regex pattern
assert regex("hello world", "") == 0.0  # Empty pattern matches anything
assert regex("", ".*") == 0.0  # Empty string matches wildcard
assert regex("", "") == 0.0  # Empty string matches empty pattern

# Case sensitivity
assert regex("Hello World", "hello") == 0.0  # Case-insensitive by default



## Running Example

In [ ]:
example_yaml = """
input:
  content: "hello world"
expected:
  # we give the name of the trace node
  - node_a:
      # we describe what output we expect from the node using accessors as keys
      # the value is what we expect the accessor to return
      b.c: |
        jimmy went
        to the store
      # we can also give a label to the node so we can refer to it later
      # using the $label key
      $label: node_a1

  - node_b:
      # we can give multiple comparisons to the same node, using different accessors
      d.e:
        value: jimmy
        comparison: "regex"
      f.g:
        value: "is a good boy"
        comparison: "chat"
        kwargs:
          case_sensitive: false

  # we can also give a regex to match the node name
  - node_.*:
      .: "store"
      # using the $parallel key we can specify that this node is expected in parallel with the previous node
      # so we do not know which trace will be logged first
      $parallel: true
      $label: node_z

  - node_c:
      b.c: "store"
      # we can specify more complex ordering constraints using before and after using the $label key
      # before and after are either a label or a list of labels
      # in this case we say that node_c should be after node_a1 and before node_z
      $after: node_a1
      $before: node_z
      
"""



In [ ]:
example_trace = [
    {
        # should be ignored
        "name": "Start",
        "output": "hello world"
    },
    {
        "name": "node_a",
        "output": {'b':{'c':"jimmy went\nto the store\nto buy some milk"}}
    },
    {
        # first option to node c
        "name": "node_c",
        "output": {'b':{'c':"store is good"}}
    },
    {
        # shouldnt match
        "name": "node_a2",
        "output": {'b':{'d':"store"}}
    },
    {
        # first option to node_z
        "name": "node_x",
        "output": "store"
    },
    {
        "name": "node_b",
        "output": {
            'f':{'g':"is a good boy"},
            'd':{'e':"jimmy"}
            }
    },
    {   
        # second option to node c, only relevant if node_* matches to node_y
        "name": "node_c",
        "output": {'b':{'c':"store is good but not good enough"}}
    },
    {
        # second option to node_z
        "name": "node_y",
        "output": "stores"
    },
]

# expected labels: [noda_a1,1 (b) ,node_z,3 (c)] # c needs to be before z
# a:[1], c:[2,6] , b:[5] z:[4,7]

In [ ]:
possible_mappings = {
    frozendict({'node_a1':1,'1':5,'node_z':4,'3':2}), 
    frozendict({'node_a1':1,'1':5,'node_z':7,'3':2}),
    frozendict({'node_a1':1,'1':5,'node_z':7,'3':6}),
}

best_mapping = frozendict({'node_a1':1,'1':5,'node_z':4,'3':2})

## Parsing Expected Traces

In [ ]:
#| export 
from typing import Dict, Any,Optional, Union, List
from pathlib import Path
from pprint import pprint
import yaml

In [ ]:
#| export
class Condition(BaseModel):
    accessor: tuple[str, ...]
    value: Any
    comparison: Optional[str] = None
    kwargs: Dict[str,Any] = {}

class ExpectedTraceStep(BaseModel):
    name: str
    label: Union[str,int]
    conditions: List[Condition]
    before: Optional[List[Union[str,int]]] = None
    after: Optional[List[Union[str,int]]] = None

class ExpectedTrace(BaseModel):
    input: List[Any]
    expected: List[ExpectedTraceStep]

class Trace(BaseModel):
    model_config = ConfigDict(extra='allow')
    name: str
    output: Any

In [ ]:
#| export
def parse_expected_trace_step(yaml_obj: Dict[str,Any],idx:int,labels:List[str]) -> ExpectedTraceStep:
    if len(yaml_obj.keys()) != 1:
        raise SyntaxError(f"Expected a single key in trace step {idx}, got {yaml_obj.keys()}")
    
    name = list(yaml_obj.keys())[0]
    value = yaml_obj[name]
    label = value.pop("$label",None)
    if label is None:
        label = str(idx)

    before = value.pop("$before",list())
    if isinstance(before,str):
        before = [before]
    after = value.pop("$after",list())
    if isinstance(after,str):
        after = [after]
    parallel = value.pop("$parallel",False)

    if parallel and idx == 0:
        raise ValueError(f"Expected trace step {idx} is has $parallel: true, but is the first step")

    if not parallel and len(after) == 0 and idx > 0:
        after.append(labels[-1])
    
    conditions = []
    for accessor,params in value.items():
        if isinstance(params,str):
            params = {"value":params}
        try:
            accessor = parse_edge_descriptor(accessor,start='accessor')
        except Exception as e:
            raise SyntaxError(f"Error parsing accessor {accessor} for step {idx}. Make sure it is formatted correctly") from e
        condition_data ={
            'accessor':accessor,
            **params
        }
        try:
            conditions.append(Condition.model_validate(condition_data))
        except Exception as e:
            raise SyntaxError(f"When parsing condition {value} for step {idx}") from e
    
    return ExpectedTraceStep(name=name,label=label,conditions=conditions,before=before,after=after)
        
    

In [ ]:
yaml_obj = yaml.safe_load(example_yaml)
sub_yaml = yaml_obj['expected'][-1]
sub_yaml

{'node_c': {'b.c': 'store', '$after': 'node_a1', '$before': 'node_z'}}

In [ ]:
parsed =  parse_expected_trace_step(sub_yaml,0,labels=[])
expected = ExpectedTraceStep(name='node_c', label="0", conditions=[Condition(accessor=('b','c'), value='store', comparison=None, kwargs={})], before=['node_z'], after=['node_a1'])
assert parsed == expected, parsed

In [ ]:
parsed_traces = [Trace.model_validate(trace) for trace in example_trace]
parsed_traces

[Trace(name='Start', output='hello world'),
 Trace(name='node_a', output={'b': {'c': 'jimmy went\nto the store\nto buy some milk'}}),
 Trace(name='node_c', output={'b': {'c': 'store is good'}}),
 Trace(name='node_a2', output={'b': {'d': 'store'}}),
 Trace(name='node_x', output='store'),
 Trace(name='node_b', output={'f': {'g': 'is a good boy'}, 'd': {'e': 'jimmy'}}),
 Trace(name='node_c', output={'b': {'c': 'store is good but not good enough'}}),
 Trace(name='node_y', output='stores')]

In [ ]:
#| export
def parse_expected_trace(yaml_str: str) -> ExpectedTrace:
    if isinstance(yaml_str,Path):
        yaml_string = yaml_str.read_text()
    else:
        yaml_string = yaml_str
    
    try:
        yaml_obj = yaml.safe_load(yaml_string)
    except Exception as e:
        raise SyntaxError(f"Error parsing yaml:\n{yaml_string}\n{e}")

    if list(yaml_obj.keys()) != ["input","expected"]:
        raise SyntaxError(f"Expected keys in main scope are 'input' and 'expected', got {yaml_obj.keys()}")

    input = yaml_obj["input"]
    if not isinstance(input,list):
        input = [input]
    expected = yaml_obj["expected"]

    parsed_steps = []
    labels = []
    for i,expected_step in enumerate(expected):
        try:
            step = parse_expected_trace_step(expected_step,i,labels)
            parsed_steps.append(step)
            labels.append(step.label)
        except Exception as e:
            raise SyntaxError(f"Error parsing expected trace step:\n{expected_step}") from e
    return ExpectedTrace(input=input,expected=parsed_steps)



In [ ]:
parsed_expected = parse_expected_trace(example_yaml)

expected = parsed_expected.expected
expected

[ExpectedTraceStep(name='node_a', label='node_a1', conditions=[Condition(accessor=('b', 'c'), value='jimmy went\nto the store\n', comparison=None, kwargs={})], before=[], after=[]),
 ExpectedTraceStep(name='node_b', label='1', conditions=[Condition(accessor=('d', 'e'), value='jimmy', comparison='regex', kwargs={}), Condition(accessor=('f', 'g'), value='is a good boy', comparison='chat', kwargs={'case_sensitive': False})], before=[], after=['node_a1']),
 ExpectedTraceStep(name='node_.*', label='node_z', conditions=[Condition(accessor=('.',), value='store', comparison=None, kwargs={})], before=[], after=[]),
 ExpectedTraceStep(name='node_c', label='3', conditions=[Condition(accessor=('b', 'c'), value='store', comparison=None, kwargs={})], before=['node_z'], after=['node_a1'])]

## Dynamic Partial Time Warping

In [ ]:
#| export
from stringdale.core import maybe_await

In [ ]:
#| export
async def compute_trace_distance(trace,expected,comparisons,default_comparison):

    logger.debug(f"Computing distance for trace {trace} and expected {expected}")
    if not re.search(expected.name, trace.name):
        return None,[]
    
    # check if all accessors are in the trace
    for condition in expected.conditions:
        try: 
            sub_object = access_object(trace.output,condition.accessor)
        except Exception as e:
            return None, []

    distance = 0
    debug_info = []
    for condition in expected.conditions:
        condition_func = comparisons.get(condition.comparison, default_comparison)
        output_sub_value = access_object(trace.output,condition.accessor)
        try:
            condition_distance = await maybe_await(condition_func,args=[output_sub_value, condition.value],kwargs=condition.kwargs)
        except Exception as e:
            raise ValueError(f"Error computing distance for condition {condition} on trace {trace.name}: {e}") from e
        distance += condition_distance
        debug_info.append({
            "comparison": condition_func.__qualname__,
            "kwargs": condition.kwargs,
            "expected": condition.value,
            "actual": output_sub_value,
            "distance": condition_distance,
        })
    
    return distance,debug_info


In [ ]:
#| export
async def compute_distances(
    traces_outputs:List[Any],
    expected_trace:ExpectedTrace,
    comparisons:Dict[str,Callable],
    default_comparison:Callable):
    """
    Compute the distance matrix between the traces and the expected traces.

    Args:
        traces_outputs: List[Any], the outputs of the traces
        expected_traces: ExpectedTrace, the expected traces
        comparisons: Dict[str,Callable], the comparisons to use for the distance matrix
        default_comparison: Callable, the default comparison to use for the distance matrix
    """
    expected_steps = expected_trace.expected
    distances = defaultdict(dict)
    debug_info = defaultdict(dict)
    
    a_iter = list(it.product(enumerate(traces_outputs), enumerate(expected_steps)))
    tasks = [
        compute_trace_distance(trace,expected,comparisons,default_comparison)
        for (i, trace), (j, expected) in a_iter
    ]
    distance_list = await asyncio.gather(*tasks)
    
    for ((i, trace), (j, expected)), (d,debug) in zip(a_iter, distance_list):
        if not d == None:
            if not d == np.inf:
                distances[expected.label][i] = d
            debug_info[expected.label][i]={
                'comparisons':debug,
                'distance':d,
                'expected_idx':j,
                'actual_idx':i,
                'actual_name':trace.name,
                'expected_name':expected.name,
                'expected_label':expected.label,
            }

    return dict(distances),dict(debug_info)
    

In [ ]:
parsed_traces

[Trace(name='Start', output='hello world'),
 Trace(name='node_a', output={'b': {'c': 'jimmy went\nto the store\nto buy some milk'}}),
 Trace(name='node_c', output={'b': {'c': 'store is good'}}),
 Trace(name='node_a2', output={'b': {'d': 'store'}}),
 Trace(name='node_x', output='store'),
 Trace(name='node_b', output={'f': {'g': 'is a good boy'}, 'd': {'e': 'jimmy'}}),
 Trace(name='node_c', output={'b': {'c': 'store is good but not good enough'}}),
 Trace(name='node_y', output='stores')]

In [ ]:
comparisons = {
    "regex": regex,
    "word_overlap": word_overlap,
}
default_comparison = word_overlap
# with checkLogs():
dist,debug_info = await compute_distances(parsed_traces,parsed_expected,comparisons,default_comparison)
dist

{'node_a1': {1: 0.375},
 '3': {2: 0.6666666666666667, 6: 0.8333333333333334},
 'node_z': {4: 0.0, 7: 1.0},
 '1': {5: 0.0}}

In [ ]:
debug_info

{'node_a1': {1: {'comparisons': [{'comparison': 'word_overlap',
     'kwargs': {},
     'expected': 'jimmy went\nto the store\n',
     'actual': 'jimmy went\nto the store\nto buy some milk',
     'distance': 0.375}],
   'distance': 0.375,
   'expected_idx': 0,
   'actual_idx': 1,
   'actual_name': 'node_a',
   'expected_name': 'node_a',
   'expected_label': 'node_a1'}},
 'node_z': {1: {'comparisons': [{'comparison': 'word_overlap',
     'kwargs': {},
     'expected': 'store',
     'actual': {'b': {'c': 'jimmy went\nto the store\nto buy some milk'}},
     'distance': inf}],
   'distance': inf,
   'expected_idx': 2,
   'actual_idx': 1,
   'actual_name': 'node_a',
   'expected_name': 'node_.*',
   'expected_label': 'node_z'},
  2: {'comparisons': [{'comparison': 'word_overlap',
     'kwargs': {},
     'expected': 'store',
     'actual': {'b': {'c': 'store is good'}},
     'distance': inf}],
   'distance': inf,
   'expected_idx': 2,
   'actual_idx': 2,
   'actual_name': 'node_c',
   'expec

In [ ]:
expected_dist = {'node_a1': {1: 0.375},
 '3': {2: 0.6666666666666667, 6: 0.8333333333333334},
 'node_z': {4: 0.0, 7: 1.0},
 '1': {5: 0.0}}

assert dist == expected_dist

In [ ]:
parsed_expected.expected

[ExpectedTraceStep(name='node_a', label='node_a1', conditions=[Condition(accessor=('b', 'c'), value='jimmy went\nto the store\n', comparison=None, kwargs={})], before=[], after=[]),
 ExpectedTraceStep(name='node_b', label='1', conditions=[Condition(accessor=('d', 'e'), value='jimmy', comparison='regex', kwargs={}), Condition(accessor=('f', 'g'), value='is a good boy', comparison='chat', kwargs={'case_sensitive': False})], before=[], after=['node_a1']),
 ExpectedTraceStep(name='node_.*', label='node_z', conditions=[Condition(accessor=('.',), value='store', comparison=None, kwargs={})], before=[], after=[]),
 ExpectedTraceStep(name='node_c', label='3', conditions=[Condition(accessor=('b', 'c'), value='store', comparison=None, kwargs={})], before=['node_z'], after=['node_a1'])]

In [ ]:
dist = dist
expected_traces = parsed_expected.expected
traces = parsed_traces


# get a mapping between expected labels and fresh var names
label_to_var = LabelToVar()
for idx,expected_step in enumerate(parsed_expected.expected):
    label_to_var.add_label(expected_step.label,idx)



In [ ]:
#| export
def get_possible_mappings(dist,expected_traces:ExpectedTrace,label_to_var:LabelToVar):
    """
    Gets possible mappings between expected traces and actual traces.
    By building a constraint satisfaction problem and solving it.
    """
    p = Problem()
    for col_idx,expected_step in enumerate(expected_traces.expected):
        viable_trace_row_nums = list(dist[expected_step.label].keys())
        var_name = label_to_var.get_col(expected_step.label)
        p.addVariable(var_name,viable_trace_row_nums)
        logger.debug(f"Adding variable {var_name} with domain {viable_trace_row_nums}")

        for before_label in expected_step.before:
            before_var_name = label_to_var.get_col(before_label)
            logger.debug(f"Adding constraint {before_var_name} < {var_name}")
            p.addConstraint(f"{var_name} < {before_var_name}")

        for after_label in expected_step.after:
            after_var_name = label_to_var.get_col(after_label)
            logger.debug(f"Adding constraint {var_name} < {after_var_name}")
            p.addConstraint(f"{after_var_name} < {var_name}")

    # these solutions use colnames    
    solutions = p.getSolutions()
    # invert the colnames back to labels
    labeled_solutions = set(frozendict({label_to_var.get_label(k):v for k,v in sol.items()}) for sol in solutions)
    return labeled_solutions

In [ ]:
from deepdiff import DeepDiff

In [ ]:
with checkLogs():
    labeled_solutions = get_possible_mappings(dist,parsed_expected,label_to_var)

assert DeepDiff(labeled_solutions,possible_mappings) == {}
labeled_solutions

__main__ - DEBUG - Adding variable a with domain [1]
__main__ - DEBUG - Adding variable b with domain [5]
__main__ - DEBUG - Adding constraint b < a
__main__ - DEBUG - Adding variable c with domain [4, 7]
__main__ - DEBUG - Adding variable d with domain [2, 6]
__main__ - DEBUG - Adding constraint c < d
__main__ - DEBUG - Adding constraint d < a


{frozendict.frozendict({'node_a1': 1, '3': 2, '1': 5, 'node_z': 4}),
 frozendict.frozendict({'node_a1': 1, '3': 2, '1': 5, 'node_z': 7}),
 frozendict.frozendict({'node_a1': 1, '3': 6, '1': 5, 'node_z': 7})}

In [ ]:
#| export
def get_best_mapping(dist_matrix,possible_mappings,label_to_var):
    """
    dist_matrix: np.ndarray
    possible_mappings: list of tuples
    label_to_var: dict
    """
    
    score_per_solution = {}
    for sol in possible_mappings:
        sum_dist = 0
        for expected_label,trace_idx in sol.items():
            sum_dist += dist_matrix[expected_label][trace_idx]
        score_per_solution[sol] = sum_dist

    best_solution =  min(score_per_solution,key=score_per_solution.get)
    best_solution_score = score_per_solution[best_solution]
    return best_solution,best_solution_score

In [ ]:
best_mapping,best_score = get_best_mapping(dist,possible_mappings,label_to_var)
assert best_mapping == frozendict({'node_a1': 1,'3': 2,'node_z': 4,'1': 5})

In [ ]:
#| export
async def align_traces(traces_outputs,expected_trace,comparisons,default_comparison):
    """
    Compute the distance matrix between the traces and the expected traces.
    """
    label_to_var = LabelToVar()
    for idx,expected_step in enumerate(expected_trace.expected):
        label_to_var.add_label(expected_step.label,idx)

    dist,debug_info = await compute_distances(traces_outputs,expected_trace,comparisons,default_comparison)
    possible_mappings = get_possible_mappings(dist,expected_trace,label_to_var)
    best_mapping,best_score = get_best_mapping(dist,possible_mappings,label_to_var)
    return best_mapping, best_score, debug_info



## End to end test

In [ ]:
parsed_expected

ExpectedTrace(input=[{'content': 'hello world'}], expected=[ExpectedTraceStep(name='node_a', label='node_a1', conditions=[Condition(accessor=('b', 'c'), value='jimmy went\nto the store\n', comparison=None, kwargs={})], before=[], after=[]), ExpectedTraceStep(name='node_b', label='1', conditions=[Condition(accessor=('d', 'e'), value='jimmy', comparison='regex', kwargs={}), Condition(accessor=('f', 'g'), value='is a good boy', comparison='chat', kwargs={'case_sensitive': False})], before=[], after=['node_a1']), ExpectedTraceStep(name='node_.*', label='node_z', conditions=[Condition(accessor=('.',), value='store', comparison=None, kwargs={})], before=[], after=[]), ExpectedTraceStep(name='node_c', label='3', conditions=[Condition(accessor=('b', 'c'), value='store', comparison=None, kwargs={})], before=['node_z'], after=['node_a1'])])

In [ ]:
parsed_expected = parse_expected_trace(example_yaml)
example_trace

default_comparison = word_overlap
comparisons = {
    "regex": regex,
    "word_overlap": word_overlap,
}


aligned_traces,score,debug_info = await align_traces(parsed_traces,parsed_expected,comparisons,default_comparison)
assert aligned_traces == best_mapping
aligned_traces,score

(frozendict.frozendict({'node_a1': 1, '3': 2, '1': 5, 'node_z': 4}),
 1.0416666666666667)

In [ ]:
debug_info.keys()

dict_keys(['node_a1', 'node_z', '3', '1'])

## export

In [ ]:
# |hide
import nbdev; nbdev.nbdev_export()